In [1]:
#import the function from functions folder
from functions.functions import get_DataFrame, get_Normalized_DataFrame

# import the libraries
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# read the json file
data = get_DataFrame('matches/matches_england.json')

print(f"{len(data)} Matches") 

380 Matches


### Representation of the data without normalizing

In [3]:
df = pd.DataFrame(data)
df.head()

,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,referees,duration,competitionId
0,Played,4405654,38,"{'1646': {'scoreET': 0, 'coachId': 8880, 'side...",181150,2018-05-13 14:00:00,1659,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385705, 'role': 'referee'}, {'r...",Regular,364
1,Played,4405654,38,"{'1628': {'scoreET': 0, 'coachId': 8357, 'side...",181150,2018-05-13 14:00:00,1628,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381851, 'role': 'referee'}, {'r...",Regular,364
2,Played,4405654,38,"{'1609': {'scoreET': 0, 'coachId': 7845, 'side...",181150,2018-05-13 14:00:00,1609,The John Smith's Stadium,2500091,"Huddersfield Town - Arsenal, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384965, 'role': 'referee'}, {'r...",Regular,364
3,Played,4405654,38,"{'1651': {'scoreET': 0, 'coachId': 8093, 'side...",181150,2018-05-13 14:00:00,1612,Anfield,2500092,"Liverpool - Brighton & Hove Albion, 4 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385704, 'role': 'referee'}, {'r...",Regular,364
4,Played,4405654,38,"{'1644': {'scoreET': 0, 'coachId': 93112, 'sid...",181150,2018-05-13 14:00:00,1611,Old Trafford,2500093,"Manchester United - Watford, 1 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381853, 'role': 'referee'}, {'r...",Regular,364


### Manually flatten the data

In [4]:
allTeams = []
formationPlayers = []
subPlayers = []
allReferees = []

for match in data:                                                              # loop through each match
    matchTeamsData = match['teamsData']                                         # get the teams data for each match

    for team in matchTeamsData:                                                 # loop through each team in the match
        
        teamGameStats = matchTeamsData[team]                                    # get the stats for each team in the match
        teamList = list(teamGameStats.values())                                 # get the values for each teams stats in the match and convert to list
        teamList.insert(0, match['wyId'])                                  # insert the match id at the beginning of the list
        allTeams.append(teamList)

        if teamGameStats['hasFormation'] == True:
            for formationBreak in teamGameStats['formation']:                   # loop through each formation break in the match (bench, lineup, subs)
                eachFormBreak = teamGameStats['formation'][formationBreak]
                if eachFormBreak == 'null':                                     # if the formation break is null, skip it
                    continue
                elif formationBreak == 'substitutions':
                    for eachSub in eachFormBreak:                               # loop through each substitution in the match
                        vals = eachSub.values()
                        valList = list(vals)
                        valList.insert(0, f"{match['wyId']}")                   # insert the match id at the beginning of the list
                        if len(valList) < 5:                                    # if the list is less than 6, insert a null value to fill assists
                            valList.insert(2, np.nan)
                        subPlayers.append(valList)
                    continue
                else:
                    for eachPlayer in eachFormBreak:                            # loop through each player in the formation breaks (bench, lineup)
                        vals = eachPlayer.values()
                        valList = list(vals)
                        valList.insert(0, f"{formationBreak}")                  # insert the formation type at the beginning of the list
                        valList.insert(0, f"{match['wyId']}")                   # insert the match id at the beginning of the list
                        if len(valList) < 8:                                    # if the list is less than 8, insert a null value to fill assists
                            valList.insert(3, np.nan)
                        formationPlayers.append(valList)

    referees = match['referees']                                                # get the referees for each match
    refereeList = []
    refereeList.append(int(match['wyId']))                                      # insert the match id at the beginning of the list
    for referee in referees:                                                    # loop through each referee in the match
        refereeList.append(int(referee['refereeId']))

    if len(refereeList) < 7:                                                    # if the list is less than 7, insert a NaN to fill the list
        refereeList.extend([np.nan] * (7 - len(refereeList)))

    allReferees.append(refereeList)

### Players that were in the lineup and bench

In [5]:
players_df = pd.DataFrame(formationPlayers)
players_df.rename(columns={0: 'matchId', 1: 'formation', 2: 'playerId', 3: 'assists', 4: 'ownGoals', 5: 'redCards', 6: 'goals', 7: 'yellowCards'}, inplace=True)
players_df.replace('null', np.nan, inplace=True)
print(players_df.shape)
players_df[['matchId', 'assists', 'ownGoals', 'redCards', 'goals', 'yellowCards']] = pd.DataFrame({
    k: pd.to_numeric(v, errors='coerce') for k, v in players_df[['matchId', 'assists', 'ownGoals', 'redCards', 'goals', 'yellowCards']].items()}, dtype='Int64')
# players with out any red cards or yellow cards are shown 0, but time 0 makes the calculation wrong, so we replace it with NaN
players_df[['redCards', 'yellowCards']] = players_df[['redCards', 'yellowCards']].replace(0, np.nan)
players_df.describe()



(13668, 8)


,matchId,playerId,assists,ownGoals,redCards,goals,yellowCards
count,13668.0,13668.000000,0.0,13668.0,39.0,6241.0,1180.0
mean,2499908.446005,71222.289069,<NA>,0.222271,61.102564,0.158468,60.113559
std,109.714142,111915.842517,<NA>,0.627224,26.344674,0.414535,24.618287
min,2499719.0,36.000000,<NA>,0.0,10.0,0.0,2.0
25%,2499813.0,8186.000000,<NA>,0.0,38.0,0.0,42.0
50%,2499908.0,11152.000000,<NA>,0.0,68.0,0.0,63.0
75%,2500003.0,70965.000000,<NA>,0.0,85.5,0.0,82.0
max,2500098.0,532949.000000,<NA>,2.0,99.0,4.0,97.0


In [6]:
players_df.sort_values(by='goals', ascending=False).head()

,matchId,formation,playerId,assists,ownGoals,redCards,goals,yellowCards
4135,2499982,lineup,8325,<NA>,0,<NA>,4,<NA>
2768,2500023,lineup,120353,<NA>,0,<NA>,4,<NA>
6807,2499916,lineup,8717,<NA>,2,<NA>,3,<NA>
8636,2499856,lineup,7944,<NA>,0,<NA>,3,<NA>
9474,2499829,lineup,11669,<NA>,0,<NA>,3,<NA>


### Substitution details

In [7]:
subs_df = pd.DataFrame(subPlayers)
subs_df.rename(columns={0: 'matchId', 1: 'playerIn', 2: 'assists', 3: 'playerOut', 4: 'minute'}, inplace=True)
print(subs_df.shape)
subs_df.head()


(2083, 5)


,matchId,playerIn,assists,playerOut,minute
0,2500089,9127,NaN,9206,61
1,2500089,9179,NaN,93,80
2,2500089,532949,NaN,8284,94
3,2500089,7989,NaN,259531,59
4,2500089,11669,NaN,245813,59


In [8]:
subs_df.describe()

,playerIn,assists,playerOut,minute
count,2083.000000,0.0,2083.000000,2083.000000
mean,95287.219395,NaN,63549.023524,71.154585
std,128702.674150,NaN,102439.832384,14.546131
min,36.000000,NaN,38.000000,3.000000
25%,8384.000000,NaN,8284.000000,64.000000
50%,20505.000000,NaN,12536.000000,74.000000
75%,145692.000000,NaN,70085.500000,82.000000
max,532949.000000,NaN,486252.000000,95.000000


### Team details

In [9]:
team_df = pd.DataFrame(allTeams)
team_df.drop([8], axis=1, inplace=True)
team_df.rename(columns={0: 'matchId', 1: 'scoreET', 2: 'coachId', 3: 'side', 4: 'teamId', 5: 'score', 6: 'scoreP', 7: 'hasFormation', 9: 'scoreHT'}, inplace=True)
# Coach Id can not be 0, so we replace it with NaN
team_df['coachId'] = team_df['coachId'].replace(0, np.nan)
team_df.head()

matchId           int64
scoreET           int64
coachId         float64
side             object
teamId            int64
score             int64
scoreP            int64
hasFormation      int64
scoreHT           int64
dtype: object


,matchId,scoreET,coachId,side,teamId,score,scoreP,hasFormation,scoreHT
0,2500089,0,8880.0,home,1646,1,0,1,1
1,2500089,0,8934.0,away,1659,2,0,1,0
2,2500090,0,8357.0,home,1628,2,0,1,0
3,2500090,0,NaN,away,1627,0,0,1,0
4,2500091,0,7845.0,away,1609,1,0,1,1


In [10]:
team_df.describe()

,matchId,scoreET,coachId,teamId,score,scoreP,hasFormation,scoreHT
count,7.600000e+02,760.0,715.000000,760.000000,760.000000,760.0,760.0,760.000000
mean,2.499908e+06,0.0,104966.591608,2075.400000,1.339474,0.0,1.0,0.560526
std,1.097684e+02,0.0,125302.764295,1941.199054,1.275293,0.0,0.0,0.776139
min,2.499719e+06,0.0,3295.000000,1609.000000,0.000000,0.0,1.0,0.000000
25%,2.499814e+06,0.0,8357.000000,1617.500000,0.000000,0.0,1.0,0.000000
50%,2.499908e+06,0.0,14791.000000,1627.500000,1.000000,0.0,1.0,0.000000
75%,2.500003e+06,0.0,254174.000000,1644.500000,2.000000,0.0,1.0,1.000000
max,2.500098e+06,0.0,531745.000000,10531.000000,7.000000,0.0,1.0,4.000000


### Matches

In [11]:
matches_df = pd.DataFrame(data)
matches_df.drop(['teamsData'], axis=1, inplace=True)
matches_df.drop(['referees'], axis=1, inplace=True)
matches_df.rename(columns={'wyId': 'matchId'}, inplace=True)
# winning team id can not be 0, so we replace it with NaN (draw)
matches_df['winner'] = matches_df['winner'].replace(0, np.nan)
matches_df.describe()

,roundId,gameweek,seasonId,winner,matchId,competitionId
count,380.0,380.000000,380.0,279.000000,3.800000e+02,380.0
mean,4405654.0,19.500000,181150.0,1881.602151,2.499908e+06,364.0
std,0.0,10.980313,0.0,1488.850654,1.098408e+02,0.0
min,4405654.0,1.000000,181150.0,1609.000000,2.499719e+06,364.0
25%,4405654.0,10.000000,181150.0,1612.000000,2.499814e+06,364.0
50%,4405654.0,19.500000,181150.0,1624.000000,2.499908e+06,364.0
75%,4405654.0,29.000000,181150.0,1633.000000,2.500003e+06,364.0
max,4405654.0,38.000000,181150.0,10531.000000,2.500098e+06,364.0


In [12]:
matches_df.head()

,status,roundId,gameweek,seasonId,dateutc,winner,venue,matchId,label,date,duration,competitionId
0,Played,4405654,38,181150,2018-05-13 14:00:00,1659.0,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2",Regular,364
1,Played,4405654,38,181150,2018-05-13 14:00:00,1628.0,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0","May 13, 2018 at 4:00:00 PM GMT+2",Regular,364
2,Played,4405654,38,181150,2018-05-13 14:00:00,1609.0,The John Smith's Stadium,2500091,"Huddersfield Town - Arsenal, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2",Regular,364
3,Played,4405654,38,181150,2018-05-13 14:00:00,1612.0,Anfield,2500092,"Liverpool - Brighton & Hove Albion, 4 - 0","May 13, 2018 at 4:00:00 PM GMT+2",Regular,364
4,Played,4405654,38,181150,2018-05-13 14:00:00,1611.0,Old Trafford,2500093,"Manchester United - Watford, 1 - 0","May 13, 2018 at 4:00:00 PM GMT+2",Regular,364


### Referees

In [13]:
referees_df = pd.DataFrame(allReferees)
referees_df.rename(columns={0: 'matchId', 1: 'referee', 2: 'firstAssistant', 3: 'secondAssistant', 4: 'fourthOfficial', 5: 'firstAdditionalAssistant', 6: 'secondAdditionalAssistant'}, inplace=True)
referees_df.head()

,matchId,referee,firstAssistant,secondAssistant,fourthOfficial,firstAdditionalAssistant,secondAdditionalAssistant
0,2500089,385705.0,385733.0,385922.0,388333.0,NaN,NaN
1,2500090,381851.0,385011.0,385920.0,381852.0,NaN,NaN
2,2500091,384965.0,381917.0,383201.0,385737.0,NaN,NaN
3,2500092,385704.0,385919.0,456216.0,NaN,NaN,NaN
4,2500093,381853.0,386732.0,393151.0,385915.0,NaN,NaN


In [14]:
referees_df.describe()

,matchId,referee,firstAssistant,secondAssistant,fourthOfficial,firstAdditionalAssistant,secondAdditionalAssistant
count,3.800000e+02,378.000000,378.000000,378.000000,377.000000,0.0,0.0
mean,2.499908e+06,385166.134921,384511.277778,389574.888889,385578.299735,NaN,NaN
std,1.098408e+02,6088.097808,1875.845494,16402.158745,6004.503475,NaN,NaN
min,2.499719e+06,378951.000000,379551.000000,379551.000000,378951.000000,NaN,NaN
25%,2.499814e+06,381851.250000,383201.000000,385011.000000,381853.000000,NaN,NaN
50%,2.499908e+06,385704.000000,385005.000000,385918.000000,385737.000000,NaN,NaN
75%,2.500003e+06,385911.000000,385917.000000,386445.250000,385915.000000,NaN,NaN
max,2.500098e+06,408156.000000,386732.000000,456216.000000,408156.000000,NaN,NaN


## Merging matches with teams

In [18]:
matche_team_df = matches_df.merge(team_df, on='matchId', how='left')
# If matche_team_df[matche_team_df['teamId'] == matche_team_df['winner']] replace winner column with 1, else 0
matche_team_df['winner'] = np.where(matche_team_df['teamId'] == matche_team_df['winner'], 1, 0)
matche_team_df.head()

,status,roundId,gameweek,seasonId,dateutc,winner,venue,matchId,label,date,duration,competitionId,scoreET,coachId,side,teamId,score,scoreP,hasFormation,scoreHT
0,Played,4405654,38,181150,2018-05-13 14:00:00,0,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2",Regular,364,0,8880.0,home,1646,1,0,1,1
1,Played,4405654,38,181150,2018-05-13 14:00:00,1,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2",Regular,364,0,8934.0,away,1659,2,0,1,0
2,Played,4405654,38,181150,2018-05-13 14:00:00,1,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0","May 13, 2018 at 4:00:00 PM GMT+2",Regular,364,0,8357.0,home,1628,2,0,1,0
3,Played,4405654,38,181150,2018-05-13 14:00:00,0,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0","May 13, 2018 at 4:00:00 PM GMT+2",Regular,364,0,NaN,away,1627,0,0,1,0
4,Played,4405654,38,181150,2018-05-13 14:00:00,1,The John Smith's Stadium,2500091,"Huddersfield Town - Arsenal, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2",Regular,364,0,7845.0,away,1609,1,0,1,1


In [23]:
teams_df_norm = get_Normalized_DataFrame('data/teams.json')
teams_df_norm.rename(columns={'wyId': 'teamId'}, inplace=True)
teams_df_norm.head()

,city,name,teamId,officialName,type,area.name,area.id,area.alpha3code,area.alpha2code
0,Newcastle upon Tyne,Newcastle United,1613,Newcastle United FC,club,England,0,XEN,
1,Vigo,Celta de Vigo,692,Real Club Celta de Vigo,club,Spain,724,ESP,ES
2,Barcelona,Espanyol,691,Reial Club Deportiu Espanyol,club,Spain,724,ESP,ES
3,Vitoria-Gasteiz,Deportivo Alav\u00e9s,696,Deportivo Alav\u00e9s,club,Spain,724,ESP,ES
4,Valencia,Levante,695,Levante UD,club,Spain,724,ESP,ES


In [35]:
matche_team_ext_df = matche_team_df.merge(teams_df_norm, on='teamId', how='left')
matche_team_ext_df.head()

,status,roundId,gameweek,seasonId,dateutc,winner,venue,matchId,label,date,...,hasFormation,scoreHT,city,name,officialName,type,area.name,area.id,area.alpha3code,area.alpha2code
0,Played,4405654,38,181150,2018-05-13 14:00:00,0,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2",...,1,1,"Burnley, Lancashire",Burnley,Burnley FC,club,England,0,XEN,
1,Played,4405654,38,181150,2018-05-13 14:00:00,1,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2",...,1,0,"Bournemouth, Dorset",AFC Bournemouth,AFC Bournemouth,club,England,0,XEN,
2,Played,4405654,38,181150,2018-05-13 14:00:00,1,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0","May 13, 2018 at 4:00:00 PM GMT+2",...,1,0,London,Crystal Palace,Crystal Palace FC,club,England,0,XEN,
3,Played,4405654,38,181150,2018-05-13 14:00:00,0,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0","May 13, 2018 at 4:00:00 PM GMT+2",...,1,0,West Bromwich,West Bromwich Albion,West Bromwich Albion FC,club,England,0,XEN,
4,Played,4405654,38,181150,2018-05-13 14:00:00,1,The John Smith's Stadium,2500091,"Huddersfield Town - Arsenal, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2",...,1,1,London,Arsenal,Arsenal FC,club,England,0,XEN,


In [38]:
matche_team_ext_df.columns

Index(['status', 'roundId', 'gameweek', 'seasonId', 'dateutc', 'winner',
       'venue', 'matchId', 'label', 'duration', 'competitionId', 'scoreET',
       'coachId', 'side', 'teamId', 'score', 'scoreP', 'scoreHT',
       'officialName', 'area.id'],
      dtype='object')

In [37]:
matche_team_ext_df.drop(['hasFormation', 'area.name', 'area.alpha3code', 'area.alpha2code','name', 'city', 'type', 'date'], axis=1, inplace=True)
matche_team_ext_df.head()

,status,roundId,gameweek,seasonId,dateutc,winner,venue,matchId,label,duration,competitionId,scoreET,coachId,side,teamId,score,scoreP,scoreHT,officialName,area.id
0,Played,4405654,38,181150,2018-05-13 14:00:00,0,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2",Regular,364,0,8880.0,home,1646,1,0,1,Burnley FC,0
1,Played,4405654,38,181150,2018-05-13 14:00:00,1,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2",Regular,364,0,8934.0,away,1659,2,0,0,AFC Bournemouth,0
2,Played,4405654,38,181150,2018-05-13 14:00:00,1,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0",Regular,364,0,8357.0,home,1628,2,0,0,Crystal Palace FC,0
3,Played,4405654,38,181150,2018-05-13 14:00:00,0,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0",Regular,364,0,NaN,away,1627,0,0,0,West Bromwich Albion FC,0
4,Played,4405654,38,181150,2018-05-13 14:00:00,1,The John Smith's Stadium,2500091,"Huddersfield Town - Arsenal, 0 - 1",Regular,364,0,7845.0,away,1609,1,0,1,Arsenal FC,0
